In [1]:
from math import sqrt,ceil, floor

In [2]:
import numpy as np
def arbre_recomb(n,b): #arbre avec recombinaison sous forme de tableau
    l=2*b*n+1
    S=np.zeros((l,n+1))
    for i in range(1,n+1):
        for j in range(l):
            if j<b*n :
                for k in range(b*(n-i),b*n):
                    S[k,i]=b*n-k
            if j>b*n:
                S[j,i]=-S[2*b*n-j,i]
    return S

In [3]:
def arbre_recomb2(n,b):
    l=2*b*n+1
    S=np.zeros((l,n+1))
    for i in range(l):
        S[i,n]= b*n-i
    for j in range(1,n+1):
        for i in  range(l):
            if i>=b*j and i<=b*(2*n-j):
                S[i,n-j]=b*n-i
    return S


In [4]:
arbre_recomb2(2500,3)

array([[    0.,     0.,     0., ...,     0.,     0.,  7500.],
       [    0.,     0.,     0., ...,     0.,     0.,  7499.],
       [    0.,     0.,     0., ...,     0.,     0.,  7498.],
       ...,
       [    0.,     0.,     0., ...,     0.,     0., -7498.],
       [    0.,     0.,     0., ...,     0.,     0., -7499.],
       [    0.,     0.,     0., ...,     0.,     0., -7500.]])

In [5]:
def proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n):
    proba=np.zeros((3,len(List_sigma)))
    h=T/n
    for i in range(len(List_sigma)):
        proba[0,i]=(List_sigma[i]**2+List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[1,i]=(List_sigma[i]**2-List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[2,i]=1-(proba[0,i]+proba[1,i])
    return proba

In [6]:
arbre=arbre_recomb2(2500,3)

In [7]:
kappa = 3.0
theta = 0.04
sigma_v = 0.1
r = 0.05
rho = -0.1 # covariance between price and volatility brownian motions
v0 = 0.04
sigma_bar = 0.2

In [8]:
def prix_call_europeen(K,S_o,n,b,sigma_bar,T,List_vi,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp((arbre[i,n])*sigma_bar*np.sqrt(h)+(rho/sigma_v)*(List_vi[reg]-v0)+(0.05-rho*kappa*theta/sigma_v)*n*h),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [9]:
dw = 0.02

In [10]:
ml, mu=  1, 2
m = mu-ml+1
regimes = range(1,m+1)

In [11]:
# Compute the Markov chain

# upper and lower bounds for the discretization of the volatility


def transition(k):

    forward = sigma_v**2/(2*(dw)**2) + (2*kappa*theta-sigma_v**2)/(2*k*(dw)**2) - kappa*k/4

    backward = sigma_v**2/(2*(dw)**2) - (2*kappa*theta-sigma_v**2)/(2*k*(dw)**2) + kappa*k/4

    return forward,backward


def transition(k):

    forward = sigma_v**2/(2*(dw)**2) + (2*kappa*theta-(sigma_v**2)/2)/(2*k*(dw)**2) - kappa*k/4

    backward = sigma_v**2/(2*(dw)**2) - (2*kappa*theta-(sigma_v**2)/2)/(2*k*(dw)**2) + kappa*k/4

    return forward,backward


def compute_Q():
    
    Q = np.zeros((m,m))

    for i in range(1,m-1):
        
        forward,backward = transition(i+ml-1)

        if forward>=0 and backward>=0:

            Q[i,i+1] = forward
            Q[i,i-1] = backward
            Q[i,i] = -(forward+backward)

        elif forward>0:

            Q[i,i-1] = backward + kappa*(i+ml-1)/4
            Q[i,i+1] = sigma_v**2/(2*(dw)**2) 
            Q[i,i] = -forward-backward

        else:
            
            Q[i,i-1] = sigma_v**2/(2*(dw)**2) 
            Q[i,i+1] = forward - kappa*(i+ml-1)/4
            Q[i,i] = -forward-backward
    # i =0
    Q[0,1] = (2*kappa*theta-(sigma_v**2)/2)/(ml*(dw)**2) - kappa*ml/2
    Q[0,0] = - Q[0,1]
    # i = m-1
    Q[m-1,m-2] = -(2*kappa*theta-(sigma_v**2)/2)/(mu*(dw)**2) + kappa*mu/2
    Q[m-1,m-1] = - Q[m-1,m-2]

    return Q

In [12]:
def transition_matrix(Q,h):
    nbr=Q.shape[0]
    tr_matrix=np.zeros((nbr,nbr))
    for i in range(nbr):
        for j in range(nbr):
            if i==j:
                tr_matrix[i,j]=np.exp(Q[i,j]*h)
            else:
                tr_matrix[i,j]=-(1-np.exp(Q[i,i]*h))*Q[i,j]/Q[i,i]
    tr_matrix = np.nan_to_num(tr_matrix, nan=0)
    return tr_matrix


In [13]:
Q=np.array([[-0.5,0.5],[0.5,-0.5]])
P=transition_matrix(Q,1/1000)

In [14]:
Q1=compute_Q()
P1=transition_matrix(Q1,0.25/2500)

In [15]:
P1

array([[ 0.94308393,  0.05691607],
       [-0.0295018 ,  1.0295018 ]])

In [16]:
L1=[0.15,0.25]
L2=[0.05-0.5*0.15**2,0.05-0.5*0.25**2]
L3=[1,2]
L4=[0.05,0.05]

In [17]:
list_vi = [(1/4)*((i+ml-1)*dw)**2 for i in range(1,m+1)]
list_sigma = [np.sqrt(list_vi[i]*(1-rho**2)) for i in range(len(list_vi))]
list_a = [list_vi[i]*((rho*kappa/sigma_v)-0.5) for i in range(len(list_vi))]

list_saut = [0]*m
for i in range (m):
    k1,k2 = floor(2*list_sigma[i]/sigma_bar),ceil(2*list_sigma[i]/sigma_bar)
    if k1==k2 or k1*sigma_bar<list_sigma[i]:
        list_saut[i] = k2
    else:
        c = ((k1*sigma_bar)**2-list_sigma[i]**2)/list_a[i]**2
        d = (k2*sigma_bar-sqrt((k2*sigma_bar)**2-4*list_sigma[i]**2))**2/(4*list_a[i])**2
        if c<=d:
            list_saut[i] = k2
        else:
            list_saut[i] = k1

list_r = [0.05]*m

In [18]:
proba_up_middle_down(list_sigma,list_a,list_saut,0.2,0.25,2500)

array([[0.00122875, 0.004915  ],
       [0.00124625, 0.004985  ],
       [0.997525  , 0.9901    ]])

In [19]:
r=0.05
A=prix_call_europeen(100,90,2500,3,0.2,0.25,list_vi,list_sigma,list_a,list_saut,list_r,P1)

In [20]:
A[:,0][2500*3]

{0: 0.01777893549353699, 1: 0.01807342238927407}

In [60]:
def prix_put_europeen(K,S_o,n,b,sigma_bar,T,List_vi,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp((arbre[i,n])*sigma_bar*np.sqrt(h)+(rho/sigma_v)*(List_vi[reg]-v0)+(r-rho*kappa*theta/sigma_v)*n*h),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [346]:
def prix_call_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],-K+S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [347]:
def prix_put_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],K-S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix